## Loading libraries

In [ ]:
import time
import json
import os
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from datetime import datetime
from tqdm.notebook import tqdm

# Local imports
from utils.text_encoder import TextEncoder
from Random_generator import RandomGenerator, TextDecoder

logging.basicConfig(level=logging.INFO)

## Loading h_sample, h_population

In [2]:
h_sample = pd.read_csv('h_sample.csv')
h_population = pd.read_csv('h_population.csv')
print("Data loaded successfully!")

Data loaded successfully!


## Encoding h_sample with Random Ordering

In [3]:
encoder = TextEncoder()

# Encoding
train_texts = encoder.encode_dataset(h_sample)

print("Encoding h_sample is completed.\n\n Test of 'Random Ordering' in first sample (profile):")
for idx, text in enumerate(train_texts[:10], start=1):
    print("------------------------------")
    print(f"Example{idx}: \n\"{text}\"")

Encoding h_sample is completed.

 Test of 'Random Ordering' in first sample (profile):
------------------------------
Example1: 
"Work days is 5 days, Home type is Apartment, Age group is [35,40), Major travel mode is Car, Number of household members is 4, Car ownership of household is Yes, Household monthly income level is 3M-5M KRW, Gender is Male, Education status is Not student, Kid in household is Yes, Work type is Simple labor, Driver license is Yes, Major departure time is Peak."
------------------------------
Example2: 
"Kid in household is No, Household monthly income level is 1M-3M KRW, Driver license is Yes, Age group is [25,30), Major departure time is Peak, Work days is 5 days, Home type is Multi-family, Education status is Not student, Major travel mode is Public Transportation, Car ownership of household is No, Gender is Female, Work type is Manager/Office, Number of household members is 2."
------------------------------
Example3: 
"Work days is Inoccupation/non-regular

## Make the LLM-Random model: Fine-tuning the GPT-2 with random ordering

In [4]:
from torch.utils.data import Dataset, DataLoader
import time
import torch
import json
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
import pandas as pd

class DynamicTextDataset(Dataset):
    """
    For each **getitem** call, it dynamically encodes the corresponding row using TextEncoder.
    Therefore, a different valid linear extension is applied each time.
    """
    def __init__(self, dataframe, encoder):
        self.dataframe = dataframe
        self.encoder = encoder
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        return self.encoder.encode_row(row)

# Creating DataLoader (using DynamicTextDataset for dynamic encoding)
dynamic_dataset = DynamicTextDataset(h_sample, encoder)
batch_size = 8
dataloader = DataLoader(dynamic_dataset, batch_size=batch_size, shuffle=True)

# -------------------------------------------
# Training
# -------------------------------------------
print("\nFine-tuning the LLM-Random model")
generator = RandomGenerator(model_name='distilgpt2')  # gpt2-medium, gpt2-large
learning_rate = 5e-5
epochs = 40
generator.init_optimizer(learning_rate=learning_rate)

training_metrics = {
    'epoch_losses': [],
    'epoch_times': [],
    'epoch_tokens': [],
    'gpu_memory_usage': [],
    'batch_metrics': []
}

results_dir = Path('Training Metrics')
results_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

start_time = time.time()
initial_memory = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0

for epoch in range(epochs):
    epoch_start_time = time.time()
    epoch_metrics = {
        'loss': 0,
        'tokens': 0,
        'batches': []
    }
    
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{epochs}')
    for batch_idx, batch in enumerate(progress_bar):
        inputs = generator.tokenizer(
            batch, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=512
        ).to(generator.device)
        
        outputs = generator.model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss.item()
        
        generator.optimizer.zero_grad()
        outputs.loss.backward()
        generator.optimizer.step()
        
        batch_tokens = inputs['input_ids'].numel()
        epoch_metrics['batches'].append({
            'batch_idx': batch_idx,
            'loss': loss,
            'tokens': batch_tokens
        })
        epoch_metrics['tokens'] += batch_tokens
        epoch_metrics['loss'] += loss
        
        progress_bar.set_postfix({
            'batch': f'{batch_idx+1}/{len(dataloader)}',
            'loss': f'{loss:.4f}',
            'tokens': batch_tokens
        })
    
    epoch_time = time.time() - epoch_start_time
    avg_loss = epoch_metrics['loss'] / len(dataloader)
    
    training_metrics['epoch_losses'].append(avg_loss)
    training_metrics['epoch_times'].append(epoch_time)
    training_metrics['epoch_tokens'].append(epoch_metrics['tokens'])
    if torch.cuda.is_available():
        current_memory = torch.cuda.memory_allocated()
        training_metrics['gpu_memory_usage'].append((current_memory - initial_memory) / 1e6)
    training_metrics['batch_metrics'].append(epoch_metrics)
    
    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Epoch Time: {epoch_time:.2f}s")
    print(f"Tokens Processed: {epoch_metrics['tokens']:,}")
    if torch.cuda.is_available():
        current_memory = torch.cuda.memory_allocated()
        print(f"GPU Memory: {(current_memory - initial_memory)/1e6:.2f}MB")
    
    # Save model every 5 epochs
    if (epoch + 1) % 5 == 0:
        epoch_save_path = f"saved_models/LLM-Random_distilgpt2_epoch{epoch+1}"
        generator.save_model(epoch_save_path)
        print(f"Model saved to {epoch_save_path}")
        
training_metrics['total_time'] = time.time() - start_time
training_metrics['total_tokens'] = sum(training_metrics['epoch_tokens'])
training_metrics['peak_memory'] = (torch.cuda.max_memory_allocated() - initial_memory) / 1e6 if torch.cuda.is_available() else 0

print("\nTraining Complete!")
print(f"Total Training Time: {training_metrics['total_time']:.2f}s")
print(f"Total Tokens Processed: {training_metrics['total_tokens']:,}")
print(f"Peak GPU Memory Usage: {training_metrics['peak_memory']:.2f}MB")

metrics_file = results_dir / f'training_metrics_{timestamp}.json'
with open(metrics_file, 'w') as f:
    json.dump(training_metrics, f, indent=2)
print(f"\nTraining metrics saved to {metrics_file}")


Fine-tuning the LLM-Random model
Using device: cuda


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.00:00<?, ?it/s]
Epoch 1/40: 100%|████████████████████████| 6665/6665 [03:10<00:00, 35.03it/s, batch=6665/6665, loss=0.3264, tokens=294]



Epoch 1/40
Average Loss: 0.3634
Epoch Time: 190.25s
Tokens Processed: 5,123,734
GPU Memory: 1080.16MB


Epoch 2/40: 100%|████████████████████████| 6665/6665 [03:14<00:00, 34.33it/s, batch=6665/6665, loss=0.3600, tokens=276]



Epoch 2/40
Average Loss: 0.3411
Epoch Time: 194.12s
Tokens Processed: 5,123,444
GPU Memory: 1079.27MB


...


Epoch 38/40
Average Loss: 0.3326
Epoch Time: 188.47s
Tokens Processed: 5,123,410
GPU Memory: 1080.18MB


Epoch 39/40: 100%|███████████████████████| 6665/6665 [03:04<00:00, 36.05it/s, batch=6665/6665, loss=0.3418, tokens=282]



Epoch 39/40
Average Loss: 0.3325
Epoch Time: 184.91s
Tokens Processed: 5,123,962
GPU Memory: 1077.29MB


Epoch 40/40: 100%|███████████████████████| 6665/6665 [03:09<00:00, 35.25it/s, batch=6665/6665, loss=0.3305, tokens=282]



Epoch 40/40
Average Loss: 0.3325
Epoch Time: 189.07s
Tokens Processed: 5,122,466
GPU Memory: 1078.28MB
Model saved to saved_models/LLM-Random_distilgpt2_epoch40
Model saved to saved_models/LLM-Random_distilgpt2_epoch40

Training Complete!
Total Training Time: 7554.89s
Total Tokens Processed: 204,941,915
Peak GPU Memory Usage: 2153.47MB

Training metrics saved to Training Metrics\training_metrics_20250505_163534.json
